# Memo Series & Knowledgebase 


## CASA Memos 

CASA Memo Series

- [CASA Memo 1: MeasurementSet Definition version 2.0](https://drive.google.com/file/d/1IapBTsFYnUT1qPu_UK09DIFGM81EIZQr/view?usp=sharing) - 
  A.J. Kemball & M.H. Wieringa, v. 01/2000

- [CASA Memo 2: Convention for UVW calculations in CASA](https://drive.google.com/file/d/1a-eUwNrfnYjaUQTjJDfOjJCa8ZaSzZcn/view?usp=sharing) - U. Rau, v. 02/2013

- [CASA Memo 3: MeasurementSet Selection Syntax](https://drive.google.com/file/d/1LXrVu529pI_Di2-J6BJqmlXbgfihc2nf/view?usp=sharing) - S. Bhatnagar, v. 06/2015

- [CASA Memo 4: CASA Imager Paralellization: Measurement and Analysis of Runtime Performance for Continuum Imaging](https://drive.google.com/file/d/14GoBf8kiBCEITfyURnoFuTAAj49_FxXA/view?usp=sharing) - S. Bhatnagar & CASA HPC team, v. 03/2016

- [CASA Memo 5: CASA Performance on Lustre: serial vs parallel and comparison with AIPS](https://drive.google.com/file/d/1J4BxUl_AjxbAiFQwJLl8RZ5MYW3r-if_/view?usp=sharing) - B. Emonts, v. 06/2018

- [CASA Memo 6: User Survey and Helpdesk Statistics 2018](https://drive.google.com/file/d/18OHYERd11S2zZ8Hkx1h6j8e8iqsStkJs/view?usp=sharing) - B. Emonts, v. 11/2018

- [CASA Memo 7: ALMA Mosaicing Imaging Issues Prior to Cycle 6](https://drive.google.com/file/d/1F2LoWnJlRTdq-hRrlPfHfz78sfdhBRzM/view?usp=sharing) - North American ALMA Science Center (NAASC) Software Support Team, v. 09/2018. Copy of NAASC Memo 117 (credit: NAASC).

- [CASA Memo 8: Non-Amnesic CLEAN: A boxless/maskless CLEAN style deconvolution algorithm](https://drive.google.com/file/d/1atGvO-2ehMeWZk3FuoL2OMIKtkxjSYei/view?usp=sharing) - K. Golap, v. 12/2015

- [CASA Memo 9: Improvements to Multi-Scale CLEAN and Multi-Term Multi-Frequency CLEAN](https://drive.google.com/file/d/1_M3f4QNsMw2k-FpxtWSCKwXWC8YTRwgL/view?usp=sharing) - J.-W. Steeb & U. Rau, v. 07/2019

- [CASA Memo 10: Restoring/Common Beam and Flux Scaling in tclean](https://drive.google.com/file/d/1ngPiUsGDazhfno8RxV4MXuPI6SnbLgO0/view?usp=sharing) - J.-W. Steeb & U. Rau, v. 08/2019

- [CASA Memo 11: Heterogeneous Pointing Corrections in AW-Projection](https://drive.google.com/file/d/1Z2BZKTqp0hmQkYFNltY1X0YDWX2gPFG5/view?usp=sharing) - P. Jagannathan & S. Bhatnagar 10/2019



## CASAcore Memos 

CASA's underlying structure is based on [casacore](http://casacore.github.io/).  The [CASAcore Notes](http://casacore.github.io/casacore-notes/) describe some of the most fundamental properties, such as on the table system, data selection syntax, and data model definitions. 




## CASA Knowledgebase 

The CASA Knowledgebase pages provide bits of wisdom on CASA that should be preserved, and that may be of use to expert users.





### Correcting bad common beam 

This script re-calculates the common beam by discarding certain channels where the point-spread-function (PSF) or \'beam\' deviates substantially from those of the other channels.

It can happen that an MS contains one or more channels for which the point-spread-function (PSF) or 'beam' deviates a lot from those of the other channels. This can be the result of substantial flagging, missing data, or other factors. When making a data cube with *restoringbeam=\'common\'*, these \"outlier\" channels can create a common beam that does not reflect the beam for the bulk of the channels.

[This example_robust_common_beam.py script](https://colab.research.google.com/drive/1ad9GFgd58xwxAxyg4mhAzSR-pAcICjt0?usp=sharing) will correct the common beam by detecting and/or flagging outliers channels in the calculation of the common beam. The outlier channels are identified as those channels for which the area of those beams deviate from the median beam by a user-specified factor in computation of the median area beam. The script will do the following:

-   Run tclean with niter=0
-   Detect/flag outliers in chan beams
-   Use the remaining beams with ia.commonbeam() to get a new commonbeam
-   Specify this explicitly to tclean in the subsequent niter\>0 run.

The attach script primarily demonstrates the solution of  iter0 -\> calc_good_beam -\> specify_restoringbeam_in_iter1 along with tclean. If the new commonbeam is not larger than all the bad beams, then the iter0 tclean run\'s restoration step will throw warnings for all channels that cannot be convolved to the new common beam. 

The functionality provided by this script is not yet implemented in the commonbeam method of the image analysis tool (*ia.commonbeam*).

Please note that the script is based on an ALMA test-data that was used for characterizing the problem in pipeline processing at the NAASC (JIRA ticket PIPE-375). Parameters have to be adjusted for each use case, including heuristics to detect outlier channels and what to substitute the \'bad\' beams with.






### ALMA polarization: XY-phase solver avoids +-45 deg solution 

The XY-phase calibration obtained with CASA task gaical (gaintype=\'XYf+QU\') avoids to output +/- 45 degree solutions. This is a subtle consequence of the algorithm used to solve for the cross-hand phase, and should be harmless for the calibration of ALMA full-polarization data.

When using the task **gaincal** with parameter *gaintype=\'XYf+QU\'*, the solution to solve is a fit for a slope in 2D data (imag vs. real) from data that has noise is *both* dimensions.  In such cases, it is always better to fit a *shallow* (not steep) slope, so if the slope (for imag vs. real) comes out \>1.0, it flips the axes (to real vs. imag) and re-fits it (and inverts the resulting value to account for the swap).  This minimizes the effect of the real axis noise on the slope calculation. This yields far more accurate solutions when the nominal slope is very large (\>\>1.0, e.g., the data nearly parallel to the imag axis == cross-hand phase approaching +/-90 deg). 

The case of slope = 1.0 (which is cross-hand phase of 45 deg) corresponds to the slope at which to pivot the axis swap decision. When plotting the cross-hand phase solutions, a gap appears at +-45 deg (see figure). This gap is a property of the typical spacings between values in the statistical distribution of slope values.  I.e., for a sample of N points filling a distribution with a finite width, there is a characteristic *minimum* spacing between values that is some small *fraction* of the width of the distribution.  Of course, smaller spacings are not forbidden, but they are rare.  The axis swap reveals this property since all of the (nominal) slopes that are \>1.0 (cross-hand phases \>45.0 deg) are fit with swapped data and yield the inverse slope (\<1.0), and than inverted to be \>1.0 again.  The typical slopes mostly do not get arbitrarily close to exactly 1.0, so the gap appears.  This is essentially an extension of the fact that (for *any* slope), the precise exact value need not be realized in any instance in a sample of solutions for it.  E.g., in a sample of N gaussian-distributed values centered on some specific value the likelihood of any sample having that *precise* central value is vanishingly small. 

The gap should become smaller when either the noise decreases, or the number of channels (for the same noise) increases.

This feature should be harmless for the calibration of ALMA full-polarization data.

![f48ce1a412dae937209cd7488fb13550a88d2757](https://github.com/casangi/casadocs/blob/master/docs/notebooks/media/f48ce1a412dae937209cd7488fb13550a88d2757.png?raw=1){.image-inline}

 


### User tips on installing CASA on unsupported OSs 

This document provide tips from users on how to install CASA on unsupported operating systems, like LINUX Ubuntu, Debian, and Fedora.

<div class="alert alert-info">
Disclaimer: the information in this document is provided by users and not verified by the CASA team. The information does not reflect official CASA recommendations and should be used at your own risk.
</div>

 
CASA officially supports certain versions of LINUX Redhat and Mac OSX. See the [CASA Download page](https://casa.nrao.edu/casa_obtaining.shtml) for more information.

We realize that many users wish to try and run CASA on different operating systems. Below are some tips from the user community on installing CASA on unsupported platforms. CASA will not run on Windows.

- **Ubuntu or Debian** - Please see the following PDF: [Installing CASA on Unbuntu or Debian](https://drive.google.com/file/d/1bEJWtvY1qMmr1FH4BtPScyPT6F6zg0qK/view?usp=sharing)

- **Fedora** - Fedora 32 may cause CASA to crash on startup with the error *\"code for hash md5 was not found\"*. This is caused by changes to libssl.so in the compat-openssl10 package, which prevents the CASA supplied version of this library from loading. An easy fix is to replace the CASA version of libssl.so.10 with the OS version in /lib64 (i.e., libssl.so.1.0.2o).






### Wideband Mosaic Imaging and Pointing Corrections for the VLA Sky Survey 

This Knowledgebase article describes testing results for wideband mosaic imaging and pointing corrections for VLASS.

The Very Large Array Sky Survey ([VLASS](https://science.nrao.edu/vlass)) critically depends on accurate wideband mosaicing algorithms for its single epoch imaging and ultimately cumulative imaging. To a large degree, VLASS requirements have driven CASA's development of the AW-projection and related algorithms for [widefield](synthesis_imaging.ipynb#Wide-Field-Imaging) and [wideband](synthesis_imaging.ipynb#Wide-Band-Imaging) imaging in CASA. This Knowledgebase article provides complementary reports on the testing of Wideband Mosaic Imaging and Pointing Corrections for VLASS.

- [ARDG Test Report: Wideband Mosaic Imaging for VLASS](https://drive.google.com/file/d/1JZH9x0KxZTVqMe4hE-xvRgUAZiJ3wPaM/view?usp=sharing)

- [CASA development report: Pointing Corrections for VLASS](https://drive.google.com/file/d/1pq6Xx1AQAFldbWfSXC1An5zysjVnUYGq/view?usp=sharing)






### Calculation of Weights for Data with Varying Integration Time 

Knowledgebase Article: Calculation of Weights for Data with Varying Integration Time

**George Moellenbrock** *Original 18 Dec 2018, latest edited version 04 Nov 2019* 

When the nominal weights are expected to be uniform (becauseintegration time, channel bandwidth, effective Tsys, collecting area,etc. are all uniform, or the visibilities are normalized), extractingweight information from the apparent statistics of otherwise stablevisibility measurements is a simple matter of calculating the apparentsimple variance in the visibility real and imaginary parts over asufficient local sample of values. The real and imaginary partvariances should be approximately equal, and the inverse of their meanis the correct weight to assign to each of the visibility valueswithin the sample. Here, \"stable visibility\" means no *systematic*variation of amplitude or phase within the localsample. Noise-dominated visibilities are ideal; otherwise,well-calibrated data with no true visibility variation aredesirable. These conditions are also needed for the more general casedescribed below.

When the integration time (aka \"exposure\") varies within the localsample (such as can be generated by averaging of the datapost-correlation, where the number of samples per averaging bin mayvary, especially at the end of scans), we expect the correct variancefor each visibility to be inversely proportional to the netintegration time, and this complicates the calculation. It isnecessary to determine a weighted variance per unit inverseintegration time, wherein the sample weights for the variancecalculation are the per-visibility integration times, e$_i$. If the onlyreason the underlying variance differs among samples is the variableintegration time, then a uniform normalized variance estimate of thewhole sample may be obtained by scaling the residual data per sampleby the square root of their (known) integration times. Here, residualdata means any underlying visibility signal\-\--presumably the averageof the visibility samples, using nominal (proportional to integrationtime, etc.) weights\-\--has been subtracted. The simple variance of thisrescaled sample is, in effect, the variance per unit inverseintegration time.

For visibilities V$_i$, with integration times e$_i$:
 
\<var$_{norm}$\> = Sum (e$_i$ (V$_i$ - \<V\>)$^2$ / N    \[1\]

where \<V\> = Sum (w$_i$ V$_i$) / Sum (w$_i$)       \[1a\] 

and w$_i$ are the nominal data weights presumably proportional tointegration time and other relevant factors. In practice, we couldprobably just use w$_i$ = e$_i$ in equation \[1a\] since all of the otherrelevant factors witin w$_i$ are assumed constant within thesample. Note that the units of \<var$_{norm}$\> are in squared visibilityamplitude (Jy$^2$, presumably) times seconds. Note also that \<var$_{norm}$\>is essentially the simple variance of the ensemble \sqrt(e[$_i$).dV$_i$\](where dV$_i$ is (V$_i$-\<V\>)), i.e., of the residual visibilities scaledso that their noise is independent of integration time.

The normalized weight-per-unit-integration time is thus the inverse of\<var$_{norm}$\>:

W$_{norm}$ = 1/\<var$_{norm}$\>                      \[2\]

and per-datum revised weights may be calculated as:

W$_i$ = W$_{norm}$ \* e$_i$                         \[3\]

Another way of arriving at this result is to calculate a weightedvariance:

\<var\> = Sum (e$_i$ (V$_i$ - \<V\>)$^2$) / Sum(e$_i$)   \[4\]
 
which corresponds to the (simple) mean exposure time, which is:

\<e\> = Sum(e$_i$) / N                         \[5\]

The product of these yields \<var$_{norm}$\>, as above in \[1\]:

\<var$_{norm}$\> = \<var\>\<e\>                      \[6\]
 
and W$_{norm}$ may be formed and applied as in \[2\] and \[3\] above.

This calculation should be done for both real and imaginary parts ofthe visibility sample and averaged, or for both parts jointly, and \[3\]used to form the revised weights.

NB: In calculating sample variance, it is generally customary toacknowledge the loss of one degree of freedom due to use of the meanvisibility, \<V\> in the calculation. Essentially, \<V\> will have afinite error that tends to bias the resulting variance downward. Forsimple variance calculations, a factor N/(N-1) is applied to thevariance calculation to unbias it, and this factor can be significantfor modest N. Since a non-trivially weighted mean is used in the above(otherwise simple, non-weighted) variance calculation (eqn \[1\]), itmay be appropriate to consider a more carefully weighted calculationfor the N/(N-1) factor. The required factor is:

D = 1 - ( Sum(w$_i$\^2) / Sum(w$_i$)\^2 ) \[9\]

where w$_i$ are the a priori nominal weights used in \[1a\] above. This factor can be shown to equal (N-1)/N and so should be *divided* intothe \<var$_{norm}$\> result.

However, since the nominal error in the variance (and thus theweights) will be \<10% (an accuracy we are unlikely to achieve ingeneral anyway) for N\>10, and will be uniform over many sample groupsin the overall statwt execution, we assume that it is adequate to use thesimpler N/(N-1) factor, or omit it entirely.




### Bug affecting polarization visibility data in concatenated data 

Knowledgebase Article: characterization of a bug that affected polarization visibility data in concatenated data in CASA versions up to 5.6.

**George Moellenbrock** *Original 17 Apr 2020, latest edited version 12 May 2020*

CASA 5.7/6.1 fixed a bug in **concat** and **importfitsidi** that affected polarization visibility data in concatenated MSs. The problem that occurs in CASA versions earlier than 5.7/6.1 is that the cross-hands may be spuriously mis-ordered on a subset of baselines in the concatenated MS.This Knowledgebase Article describes the main effects that this bug has on concat\'d data in general, and in particular on the processing of ALMA and VLA data with CASA up to and including version 5.6.A careful analysis has revealed the following effects in concat\'d MSs in CASA versions prior to 5.7/6.1:

1. *In general,* visibility data cross-hands may be spuriously swapped on some baselines in concat\'d data when the antenna lists in the input MSs are *partially* different.  The concat task adopts the first (in time order) MS\'s antenna list for the output MS, and appends unique antennas from later MS(s), typically with larger indices than in their original MSs.  Depending on the original antenna indexing, baselines between these additional antennas and antennas that did occur in the first MS may sometimes require conjugation (reversal of the order of antennas in the baseline) to maintain internal indexing consistency within the output MS (for all baselines in an MS, the first antenna must have an index which is lower than (or same as) the index of the second antenna).   When baselines are conjugated in this way, the sign of the phase of each correlation and the UVWs must be reversed, and the cross-hands swapped (RL or XY will become LR or YX, respectively, and vice-versa).  Prior to CASA 5.7/6.1, the sign reversals were correct, but the cross-hand swap was spuriously omitted.  For successfully calibrated data (i.e., calibrated *prior* to running concat), this means that the sign of the imaginary part of the cross-hand visibilities will be incorrect, and thus the sign Stokes U (for circular feeds) or Stokes V (for linear feeds) will be incorrect on the affected baselines in concat\'d data. Since the pathology affects only the cross-hands, polarimetry calibration of concat\'d data may be adversely affected. 

For reconfigurable arrays, note that an antenna\'s particular position (pad) makes it unique, i.e., a specific physical antenna that has moved is a new unique antenna in this context.  Concatenation of *entirely* disjoint antenna lists are unaffected, since all additional antennas in the concatenation will have uniformly incremented indices, and no baseline conjugation will be required.   Certain unique cases of different antenna lists are immune to this problem, e.g., new unique antennas with already-higher indices than all other common antennas, etc.

2. The concat task initially sorts the MSs into time order (only at whole-MS granularity), so the effect cannot be controlled merely by adjusting the order in which the input MSs are specified in the concat call (i.e., there is no easy user-directed fix).

3. An implicit concat happens in importfitsidi (i.e., VLBI, typically) when specifying multiple FITS-IDI files. Here the antenna re-indexing occurs upon fill, and thus most-likely *before* calibration.  Since ordinary gain calibration uses only the parallel-hands, it will not be affected by the underlying cross-hand swap error.  However, polarimetry calibration will be affected to the extent that there are spuriously swapped cross-hands within the filled MS.  EVN observations consisting of multiple FITS-IDI will have the same antenna table in each file and are therefore unaffected by this bug.

4. Since the pathology affects only the cross-hands, purely Stokes I (total intensity) observations of any kind should not be affected (even if the cross-hand are present and some are affected, and thus technically incorrect).

5. For ALMA and VLA data, calibration typically occurs prior to any potentially pathological use of concat, and the impact will be as follows:

- ALMA:  Total intensity observations of any kind are not affected.  For successfully calibrated (per session) ALMA polarization data not subject to this pathology in the concat of multiple contiguous execblocks within each session\*, the pathology affects only Stokes V (circular polarization) when concat-ing multiple sessions subject to the baseline conjugation conditions described in item 1 above.  This is because the spurious cross-hand swap effectively sabotages only the sign of the imaginary part of the cross-hands in some baselines, i.e., the apparent Stokes V signal sampled by linear feeds.  The net effect will be to suppress the net Stokes V signal in imaging.   This presumably affects only a very tiny minority of existing ALMA observations.

<div class="alert alert-info">
* **Note:** In the course of standard scripted ALMA polarimetry calibration, the split task does not remove antennas from the ANTENNA subtable, even when they are fully flagged and keepflags=False.  The data rows are not retained in this case, but the ANTENNA subtable seen by concat remains complete.  Therefore, as long as the execblocks within the contiguous polarization session are uniform as regards to antenna population (as is intended), the polarization calibration within an individual session should not be subject to this this pathology.
</div>

- VLA: Total intensity observations of any kind are not affected.  For successfully calibrated VLA polarization observations (which typically do not require a prior concat), the pathology affects only Stokes U when concat-ing multiple observations subject to the baseline conjugation conditions described in item 1 above.  By the same logic as for ALMA, the Stokes U (cross-hand imaginary part for circular feeds) will be suppressed.  Since this affects part of the linearly polarized net response, a larger fraction of VLA cases (cf ALMA) may be affected.  The pathological condition arises when antennas are variously removed from the array (typically up to 2 or 3 at any given time) for incidental maintenance, so as to generate datasets with fewer than the full complement of 27 antennas, or when antennas move in and out of the barn (even when there are 27 antennas present in each observation), and when such disparate observations are concat\'d.    For concats of different VLA configurations, some baselines to antennas that did *not* move (typically \~12 out of 27 antennas) between the configurations will be affected, even if the total antenna lists (by name/number) have not changed.  This is because antennas that did move (only) are unique new antennas in the concat by virtue of their new positions, and some baselines between them and the stationary antennas must be conjugated in the concat. 

6. If observations are combined implicitly in imaging by specifying a list of MSs to tclean, there should be no problem, since the bug is an artifact of the mechanical combination of MSs into a single MS on disk.





### Single Dish imaging bug for EBs with different antenna positions in common antenna names and IDs 

A bug was found for imaging of single-dish data with the tasks **sdimaging** and **tsdimaging**in CASA versions 5.6 and lower. This bug could cause an inaccurate direction conversion when more than one MS is given and these data contain the same antenna with the same antenna ID but with different location, i.e., station or pad where the antenna is placed. The bug affects the brightness distribution and flux density in the combined image, since the coordinates are not correct for some fraction of the data set.

Details of this bug can be found in this [Knowledgebase Article](https://drive.google.com/file/d/1Fu8_GGwnxaoNthmwDu4z2BF6_igB-k9-/view?usp=sharing).

The bug was fixed in CASA 5.7/6.1



***





###CASA Data Repository for developers building from source

For general users, information on the CASA Data Repository can be found on the ["External Data" pages](external-data.ipynb#CASA-Data-Repository) in CASA Docs.

For the build from source, such as those the developers use, the "casadata" is taken from a list of directories given in .casa/config.py. A second set of datasets is necessary if the developer wants to run the CASA tests. The new casatestdata repository can be cloned following the instructions given in Bitbucket: https://open-bitbucket.nrao.edu/projects/CASA/repos/casatestdata/browse

For example, for developers at NRAO Charlottesville, the entries in their config.py could look like:

```
datapath=['/home/casa/data/casatestdata/','/home/casa/data/distro/']
```

or just this:

```
datapath=['/home/casa/data/casatestdata/']
```
 
where:
/home/casa/data/casatestdata/ is the new test data repository of CASA that contains all the data needed to run CASA tests.
/home/casa/data/distro/ contains only the "casadata" needed to launch CASA and from here it is packaged to be included in the casalith tarballs.

For CASA build from source, the distro data is found in the following place:

```
casa6 build from source: distro is taken from .casa/config.py under:    
datapath=['/some-directory/distro/']
```

```
casa5 build from source: distro is taken from under:    
$CASAPATH/data/
```

**Warning:** to developers regarding awproject using CASA 6.1 or earlier:

In CASA 6.2 a bug was fixed in the way that the awproject gridder in tclean calls the data repository in CASA 6. Previously tclean was looking at .casa/config.py for the location of the distro data but only considering the root data directory existence and not checking existence of the data (more specifically, antenna response data) actually used by it. As of CASA 6.2, the logic of checking the data path to look for the antenna response using the awproject gridder in tclean is as follows:

1. It looks for the antenna response data directory by constructing the full path with the root data paths from dataPath(), which returns data paths defined in config.py (for casa6).  The specific data directory to look for varies by telescope (i.e. ALMA vs VLA).

2. If 1 fails, it looks for the data in the path determined by the function, distroDataPath(), which returns the default root data directory path in the case of casalith tarballs.

3. If all above fail, it tries the path defined by CASAPATH (only relevant for casa5)



###Cube

##Cube Refactor##

A [refactor of cube imaging](https://github.com/casangi/casadocs/blob/master/docs/notebooks/media/Cube_Refactor_Docs.pdf?raw=1) in **tclean** was implemented in CASA 6.2  with the following goals:

- Parallel and serial should, within numerical precision, be equal (6.1 and previous gave slightly different results depending on the number of processors used)

- Eliminate *refconcat* cubes, which could be a performance problem in image analysis.

- Ability to restart *tclean* for cubes with different number of processors ( 6.1 and previous had to be restarted with the exact same number of processors used in the first run of tclean).

- Ability to save model visibilities in *MODEL_DATA* (or virtual) when running tclean in parallel.

- Remove the clash of *chanchunking* and parallel numprocesses (e.g multiple field cubes and chanchunking).

\\

As part of this extensive cube refactor, the following features were implemented in 6.2:

- Parallel and serial runs now use the same code and that has fixed the differences which used to be found between serial and different n-parallel runs. Serial and parallel runs now give identical results to within numerical precision.

- Refconcat cubes are no longer produced (also the directory *workingdir* is no longer made for cube runs)

- *tclean* can be restarted in parallel or serial independent of how the first run was achieved.

- For cubes, model visibilities can now be saved in parallel.

- In parallel run, a common beam can now be given (e.g *restoringbeam=’common’*). Previously, *tclean* had to be re-run in serial to restore to a single restoring beam for all channels.

- Tracking a moving source (with ephemeris tables) now works in parallel with *specmode=’cubesource’*.

- The *chanchunks* parameter has been removed, as the refactor code made it redundant.

- *parallel=True* or *parallel=False* is not considered for cube imaging. If casa is launched via the mpicasa call then cube imaging is in parallel else if it is invoked via the casa call then it is run in serial.

- Interactive *tclean* for cubes now works when launched in parallel.

- Using a psf to reset to another *psfphasecenter* for mosaic now works for serial and parallel.

- The major and minor cycles states have been dissociated when a major cycle is happening, the minor cycle code does not hold any state in memory and vice versa. This is the reason why a *selectvis* message will be seen  at every major cycle. This should reduce the amount of memory used.

\\

Details of the cube refactor efforts can be found in this [this pdf](https://github.com/casangi/casadocs/blob/master/docs/notebooks/media/Cube_Refactor_Docs.pdf?raw=1). 

## Reference Material 

Collection of relevant reference material




### AIPS-CASA Dictionary 

CASA tasks equivalent to AIPS tasks

 AIPS tasks and their corresponding CASA tasks. Note that this list is not complete and there is also not a one-to-one translation. 

   AIPS Task    CASA task/tool               Description
  ------------ ---------------------------- -----------------------------------------------------------------------------------------------------
  APROPOS      taskhelp                     List tasks with a short description of their purposes
  BLCAL        blcal                        Calculate a baseline-based gain calibration solution
  BLCHN        blcal                        Calculate a baseline-based bandpass calibration solution
  BPASS        bandpass                     Calibrate bandpasses
  CALIB        gaincal                      Calibrate gains (amplitudes and phases)
  CLCAL        applycal                     Apply calibration to data
  COMB         immath                       Combine images
  CPASS        bandpass                     Calibrate bandpasses by polynomial fitting
  CVEL         cvel/mstransform             Regid visibility spectra
  DBCON        concat/virtualconcat         Concatenate u-v datasets
  DEFAULT      default                      Load a task with default parameters
  FILLM        importvla                    Import old-format VLA data
  FITLD        importuvfits/importfitsidi   Import a u-v dataset which is in FITS format
  FITLD        importfits                   Import an image which is in FITS format
  FITTP        exportuvfits                 Write a u-v dataset to FITS format
  FITTP        exportfits                   Write an image to FITS format
  FRING        (coming soon)                Calibrate group delays and phase rates
  GETJY        fluxscale                    Determine flux densities for other cals
  GO           go                           Run a task
  HELP         help                         Display the help page for a task (also use [casa.nrao.edu/casadocs](http://casa.nrao.edu/casadocs))
  IMAGR        clean/tclean                 Image and deconvolve
  IMFIT        imfit                        Fit gaussian components to an image
  IMHEAD       vishead                      View header for u-v data
  IMHEAD       imhead                       View header for an image
  IMLIN        imcontsub                    Subtract continuum in image plane
  IMLOD        importfits                   Import a FITS image 
  IMSTAT       imstat                       Measure statistics on an image
  INP          inp                          View task parameters
  JMFIT        imfit                        Fit gaussian components to an image
  LISTR        listobs                      Print basic data
  MCAT         ls                           List image data files
  MOMNT        immoments                    Compute moments from an image
  OHGEO        imregrid                     Regrids an image onto another image's geometry
  PBCOR        impbcor/widebandpbcor        Correct an image for the primary beam
  PCAL         polcal                       Calibrate polarization
  POSSM        plotcal/plotms               Plot bandpass calibration tables
  POSSM        plotms                       Plot spectra
  PRTAN        listobs                      Print antenna locations
  PRTAN        plotants                     Plot antenna locations
  QUACK        flagdata                     Remove first integrations from scans
  RENAME       mv                           Rename an image or dataset
  RFLAG        flagdata                     Auto-flagging
  SETJY        setjy                        Set flux densities for flux cals
  SMOTH        imsmooth                     Smooth an image
  SNPLT        plotcal/plotms               Plot gain calibration tables
  SPFLG        msview                       Flag raster image of time v. channel
  SPLIT        split                        Write out u-v files for individual sources
  STATWT       statwt                       Weigh visibilities based on their noise
  TASK         inp                          Load a task with current parameters
  TGET         tget                         Load a task with parameters last used for that task
  TVALL        imview                       Display image
  TVFLG        msview                       Flag raster image of time v. baseline
  UCAT         ls                           List u-v data files
  UVFIX        fixvis                       Compute u, v, and w coordinates
  UVFLG        flagdata                     Flag data
  UVLIN        uvcontsub/mstransform        Subtract continuum from u-v data
  UVLSF        uvcontsub/mstransform        Subtract continuum from u-v data
  UVPLT        plotms                       Plot u-v data
  UVSUB        uvsub                        Subtracts model u-v data from corrected u-v data
  WIPER        plotms                       Plot and flag u-v data
  ZAP          rmtables                     Delete data files

 




### MIRIAD-CASA Dictionary 

CASA tasks equivalent to MIRIAD tasks

The Table below provides a list of common Miriad tasks, and their equivalent CASA tool or tool function names. The two packages differ in both their architecture and calibration and imaging models, and there is often not a direct correspondence. However, this index does provide a scientific user of CASA who is familiar with MIRIAD, with a simple translation table to map their existing data reduction knowledge to the new package.

In particular, note that the procedure of imaging and cleaning of visibility data between CASA and MIRIAD differs slightly. In MIRIAD the tasks invert, clean and restor are used in order to attain \"CLEAN\" images, whereas in CASA the task **clean**/**tclean** achieves the same steps as a single task.

  MIRIAD Task   CASA task/tool                                                    Description
  ------------- ----------------------------------------------------------------- ---------------------------------------------------------------------------------------------------------------------------
  atlod         importatca                                                        Import ATCA RPFITS files
  blflag        plotms/msview                                                     Interactive baseline based editor/flagger
  cgcurs        imview                                                            Interactive image analysis
  cgdisp        imview                                                            Image display, overlays
  clean         clean/tclean                                                      Clean an image
  delhd         imhead (mode=\'del\'), clearcal                                   Delete values in dataset/remove calibration tables 
  fits          importmiriad,importfits, exportfits, importuvfits, exportuvfits   FITS uv/image filler
  gethd         imhead (mode=\'get\')                                             Return values in image header
  gpboot        fluxscale                                                         Set flux density scale
  gpcal         gaincal, polcal                                                   Polarization leakage and gain calibration
  gpcopy        applycal                                                          Copy calibration tables from one source to another
  gpplt         plotcal, plotms                                                   Plot calibration solutions
  imcomb        immath                                                            Image combination
  imfit         imfit                                                             Image-plane component fitter
  impol         immath (mode=\'poli\' or \'pola\')                                Manipulate polarization images (see [example](http://www.atnf.csiro.au/computing/software/miriad/userguide/node149.html))
  imstat        imstat                                                            Image statistics
  imsub         imsubimage                                                        Extract sub-image
  invert        clean/tclean                                                      Synthesis imaging/make dirty map
  linmos        im tool (im.linearmosaic)                                         Linear mosaic combination of images
  maths         immath                                                            Calculations involving images
  mfboot        fluxscale                                                         Set flux density scale
  mfcal         bandpass, gaincal                                                 Bandpass and gain calibration
  moment        immoments                                                         Calculate image moments
  prthd         imhead, listobs, vishead                                          Print header of image or uvdata
  puthd         imhead (mode=\'put\')                                             Add/edit values in image header
  restor        clean                                                             Restore a clean component model
  selfcal       clean, gaincal, etc.                                              Selfcalibration of visibility data
  uvplt         plotms                                                            Plot visibility data
  uvspec        plotms                                                            Plot visibility spectra data
  uvsplit       split                                                             Split visibility dataset by source/frequency etc





### Dan Briggs\' Dissertation - Robust Weighting 

Dan Briggs\' Dissertation on high fidelity imaging with interferometers, including robust (\'Briggs\') weighting.

Link to [Dan Briggs' Dissertation](http://www.aoc.nrao.edu/dissertations/dbriggs/) \([pdf version](https://drive.google.com/file/d/1Q0kCOlRJwXV2aww_Dr9GnGFsq9dCWjMX/view?usp=sharing)\)





### Flux Calibrator Models 

Descriptions of flux calibrator models for flux density scaling

There are two categories of flux calibrator models available to determine flux density scales: compact extra-galactic sources and solar system objects. The models for bright extragalactic sources are described in the form of polynomial expressions for spectral flux densities and clean component images for spatial information. The flux density scales based on the solar system objects are commonly used to establish flux density scales for mm and sub-mm astronomy. These models consist of brightness temperature models and their ephemeris data.



#### Compact extragalactic sources

For the VLA, the default source models are customarily point sources defined by the 'Baars', 'Perley 90', 'Perley-Taylor 99', 'Perley-Butler 2010',  'Perley-Butler 2013' (time-variable), \'Perley-Butler 2017\' (time-variable) or 'Scaife-Heald 2012' flux density scales ('Perley-Butler 2017' is the current standard by default), or point sources of unit flux density if the flux density is unknown. \'Stevens-Reynolds 2016\' currently contains only one source, 1934-638, which is primarily used for flux calibrator for the ACTA.

The model images (CLEAN component images) are readily available in CASA for the sub set of the sources listed below. The task **setjy** provides listing of the available model images included in the CASA package\'s data directory. You can find the path to the directory containing your list of VLA Stokes I models by typing (inside CASA) [print os.getenv(\'CASAPATH\').split(\' \')[\[0\]]{.error} + \'/data/nrao/VLA/CalModels/\']. The [setjy Description Page](../api/casatasks.rst) in CASA Docs also lists the models that are available in CASA. These models can be plotted in plotms.

Alternatively, the user can provide a model image at the appropriate frequency in Jy/pixel units, typically the .model made by **clean** (which is a list of components per pixel, as required, although the restored .image is in Jy/beam). For unknown calibrators, however, the spectral flux distribution has to be explicitely specified in **setjy**. If you do not specify the correct path to a model (and you have not provided your own model), the default model of a point sources of unit flux density will be adopted. 

 

  
  
3C/Common Name | B1950 Name  | J2000 Name  | Alt. J2000 Name |  Standards
--- | --- | --- | --- | --- 
 -- | -- | -- | J0133-3629 | 9
3C48 | 0134+329 | 0137+331 | J0137+3309 | 1,2,3,4,5,6,7,9 
FORNAX X | -- | -- | J0322-3712 | 9 
3C123 | 0433+295 | 0437+296 | J0437+2940 | 2, 9
3C138 | 0518+165 | 0521+166 | J0521+1638 | 1,3,4,5,6
PICTOR A | -- | -- | J0519-4546 | 9
3C144 (TAURUS A/CRAB) | -- | -- | J0534+2200 | 9
3C147 | 0538+498 | 0542+498 | J0542+4951  | 1,3,4,5,6,7,9
3C196 | 0809+483 | 0813+482 | J0813+4813 | 1,2,7,9 
3C218(HYDRA A) | -- | -- | J0918-1205 | 9
3C274 (VIRGO A) | -- |  -- | J1230+1223 | 9
3C286 | 1328+307 | 1331+305 | J1331+3030 | 1,2,3,4,5,6,7,9
3C295 | 1409+524 | 1411+522 | J1411+5212 | 1,2,3,4,5,6,7,9
3C438 (HERCULES A) | -- | -- | J1651+0459 | 9
3C353 | -- | -- | J1720-0059 | 9
-- | 1934-638 | -- | J1939-6342 | 1,3,4,5,6,8
C380 | 1828+487 | 1829+487 | J1829+4845  | 7,9
3C405 (CYGNUS A) | -- |  -- | J1959+4044 | 9
3C444 | -- | -- | J2214-1701 | 9
3C461 (CASSIOPEIA A) | -- | -- | J2323+5848 |  9 


Standards are: (1) [Perley-Butler 2010](#perley-butler-2010), (2) [Perley-Butler 2013](#perley-butler-2013), (3) [Perley-Taylor 99](#perley-taylor-99), (4) [Perley-Taylor 95](#perley-taylor-95), (5) [Perley 90](#perley-90), (6) [Baars](#baars), (7) [Scaife-Heald 2012](#scaife-heald-2012), (8) [Stevens-Reynolds 2016](#stevens-reynolds-2016) (9) [Perley-Butler 2017](#perley-butler-2017)



>Known sources and their alternative names recognized by **setjy** task
  

ALMA also uses a few dozen compact QSO as flux standards, monitored 2-4 times a month at bands 3, 6 and 7 (90 - 345 GHz). Due to rapid variability these data are not packaged with CASA, but can be accessed via <https://almascience.eso.org/alma-data/calibrator-catalogue>

 

**Baars**
<a id='baars'></a>

The only standard to not have the year in the name. It is 1977. The models are second order polynomials in log(ν), valid between 408 MHz and 15 GHz.

Reference: Baars et al. (1977) [\[1\]](#Bibliography) with a commentary by Kellermann, K. I. (1999) [\[2\]](#Bibliography)

**Perley 90** 

This standard also includes 1934-638 from Reynolds (7/94) and 3C138 from Baars et al. (1977) [\[1\]](#Bibliography) .

Details can be found at <http://www.vla.nrao.edu/astro/calib/manual/baars.html>.

**Perley-Taylor 95** 

Perley and Taylor (1995.2); plus Reynolds (1934-638; 7/94) Details can be found at <http://www.vla.nrao.edu/astro/calib/manual/baars.html>.

**Perley-Taylor 99**

Perley and Taylor (1999.2); plus Reynolds (1934-638; 7/94) Details can be found at <http://www.vla.nrao.edu/astro/calib/manual/baars.html>.

**Perley-Butler 2010** 

A preliminary version of Perley-Butler 2013. This version also has coefficients for sources that showed some degree of variability (see Perley & Butler (2013) [\[3\]](#Bibliography)) but they are treated as the steady sources (i.e. no time dependent models are used).

**Perley-Butler 2013**

Flux scale for the constant flux sources 3C123, 3C196, 3C286, and 3C295 as well as variable sources (3C48, 3C138, and 3C147). The models for the variable sources are time-dependent.Reference: Perley & Butler (2013)  [\[3\]](#Bibliography) .

**Scaife-Heald 2012**

Low frequency, 30-300MHz, calibrators 3C48, 3C147, 3C196, 3C286, 3C295, and 3C380.

Reference: Scaife & Heald (2012)  [\[4\]](#Bibliography)

**Stevens-Reynolds 2016**

Low frequency (\<11GHz) polynomial from Reynolds and updated high frequecy polynomial from Stevens.

Reference: Partridge et al. (2016)  [\[5\]](#Bibliography)

 

**Perley-Butler 2017**
<a class="headerlink" href="#perley-butler-2017"></a>

The flux density scale of Perley-Butler 2013 extended downward to \~50 MHz. Twenty sources were drawn from the Baar, Perley-Butler 2013, and Scaife-Heald 2012. Flux scale for the constant flux sources Fornax A, 3C123, J0444-2809, Pictor A, 3C144, (Taurus A or Crab), 3C196, 3C218 (Hydra A),  3C274 (Virgo A or  M87), 3C286, 3C295,  3C348 (Hercules A), 3C353, 3C380, 3C405 (Cygnus A), 3C444, and 3C461 (Cassiopeia A) as well as variable sources (3C48, 3C138, and 3C147). The models for the variable sources are time-dependent. The frequency range valid for the model for each source is also listed below.

  Source         Valid frequency range in GHz
  -------------- ------------------------------
  J0133-3649     0.2-4
  3C48           0.05-50
  Fornax X       0.2-0.5
  3C123          0.06-50
  J0444-2809     0.2-2.0
  3C138          0.2-50
  Pictor A       0.2-4.0
  Taurus A       0.05-4.0
  3C147          0.05-50
  3C196          0.050-50
  Hydra A        0.05-12
  Virgo A        0.05-3
  3C286          0.05-50
  3C295          0.05-50
  Hercules A     0.2-12
  3C353          0.2-4
  3C380          0.05-4.0\*
  Cygnus A       0.05-12
  3C444          0.2-12
  Cassiopeia A   0.2-4

\* The corrected frequency range for 3C380 is noted here based on  B. J.  Butler 2018, private comunication (CAS-9538)Reference: Perley & Butler (2017)  [\[7\]](#Bibliography)

 

 



#### Solar System objects

The usual approach in mm and sub-mm regimes is to use models that are, to first order, thermal sources in the Solar System. Their apparent brightness varies in time with their distance from the Earth (and Sun), and orientation if they are not perfect spheres with zero obliquity. However, most of them have almost constant surface properties, so once those properties are measured their apparent brightness distributions, they can in principle be predicted for any time, given an ephemeris. Planets, in particular, have more complex spectra and effects such as atmospheric lines, magnetic fields, seasons, polar caps and surface features that need to be taken into account when they are available and significant. In CASA, the Solar System objects supported by **setjy** are available through the 'Butler-JPL-Horizons 2010', and 'Butler-JPL-Horizons 2012' standards. It is recommended to use \'Butler-JPL-Horizons 2012\' as it contains updated models. The 2012 models are described in ALMA Memo 594, which is available on [https://science.nrao.edu/facilities/alma/\~aboutALMA/Technology/ALMA_Memo_Series/alma594/abs594](https://science.nrao.edu/facilities/alma/aboutALMA/Technology/ALMA_Memo_Series/alma594/abs594) . Models can be found  by typing (in CASA) [print os.getenv(\'CASAPATH\').split(\' \')[\[0\]]{.error} + \'/data/alma/SolarSystemModels\'.]

The following objects are supported based on models from Butler-JPL-Horizons 2012, updated where necessary as mentioned under each object. Please refer ALMA Memo594 for the detailed comparisons with the models in Butler-JPL-Horizons-2010.

**Venus**

The model spans the frequencies from \~300MHz to 1THz. No atmospheric lines such as CO,H~2~O~,~ HDO, and etc are included. Modeled based on Clancy et al. (2012)  [\[6\]](#Bibliography).

**Mars**

Full implementation of the model of Rudy et al. (1987) [\[7\]](#Bibliography), tabulated as a function of time and frequency (30-1000GHz). No atmospheric lines are included.

**Jupiter**

Model for 30-1020GHz (from Glenn Orton, private communication), does not include synchrotron emission.

**Uranus**

Model for 60-1800GHz (from Glenn Orton and Raphael Moreno, private communication), contains no rings or synchrotron.

**Neptune**

Model for 2-2000 GHz (from Glenn Orton and Raphael Moreno, private communication), contains no rings or synchrotron.

**Io**

Spline interpolation of data points from 15 to 29980 GHz (references: please refer to the ALMA memo 594 Table 1).  Strongly not recommended to use for the primary flux calibrator for ALMA observations.

**Europa**

Spline interpolation of data points from 15 to 29980 GHz (references: please refer to the ALMA memo 594 Table 1).  Strongly not recommended to use for the primary flux calibrator for ALMA observations.

**Ganymede**

Spline interpolation of data points from 5 to 29980 GHz (references: please refer to the ALMA memo 594 Table 1).

**Callisto**

Spline interpolation of data points from 5 to 29980 GHz (references: please refer to the ALMA memo 594 Table 1).

**Titan**

Model from Mark Gurwell, from 53.3-­1024.1 GHz. Contains surface and atmospheric emission. The atmosphere includes N2-­N2 and N2-­CH4 Collision-­Induced Absorption (CIA), and lines from minor species CO, ^13^CO, C^18^O, HCN, H^13^CN and HC^15^N. See, e.g., Gurwell & Muhleman (2000) [\[8\]](#Bibliography); Gurwell (2004) [\[9\]](#Bibliography).

**Asteroids**

Some asteroids, namely Ceres, Pallas, Vesta, and Juno are included in the Butler-JPL-Horizons 2012. The models consists of the constant brightness temperature in frequency. For Ceres, Pallas, and Vesta, updated models based on thermophysical models (TPM) (T. Mueller, private communication) which are tabulated in time and frequency, are available for the observations taken after January 1st 2015, 0:00 UT. **setjy** task will automatically switch to the new models for the observations taken on and after that date. The TPM are also available for Lutetia but it is not advised to use for the absolute flux calibration for ALMA. Each of the tabulated models contains the flux density at 30, 80, 115, 150, 200, 230, 260, 300, 330, 360, 425, 650, 800, 950, and 1000 GHz. The time resolution is 1 hour for Ceres and 15 min for Lutetia, Pallas, and Vesta. The cubic interpolation is employed to obtain the flux densities at other frequencies.

**Ceres**

Model with a constant $T_b$ = 185K over frequencies (Moullet et al. 2010 [\[10\]](#Bibliography), Muller & Lagerros 2002 [\[11\]](#Bibliography), Redman et al. 1998 [\[12\]](#Bibliography), Altenhoff et al. 1996 [\[13\]](#Bibliography)) if time of the observations took place ($t_{obs}$) is before 2015.01.01, 0:00 UT, TPM if $t_{obs}$ $\ge$ 2015.01.01, 0:00 UT.

**Pallas**

Model with a constant $T_b$ = 189K (Chamberlain et al. 2009 [\[14\]](#Bibliography), Altenhoff et al. 1994 [\[15\]](#Bibliography)) for $t_{obs}$ $\lt$ 2015.01.01, 0:00 UT, and TPM for $t_{obs}$ $\ge$ 2015.01.01, 0:00 UT

**Vesta**

Model with a constant $T_b$ = 155K (Leyrat et al. 2012 [\[16\]](#Bibliography), Chamberlain et al. 2009 [\[14\]](#Bibliography), Redman et al. 1998 [\[12\]](#Bibliography), Altenhoff et al. 1994 [\[15\]](#Bibliography)) for $t_{obs}$ $\lt$ 2015.01.01, 0:00 UT, and TPM for $t_{obs}$ $\ge$ 2015.01.01, 0:00 UT

**Juno**

Model with a constant $T_b$ = 153K (Chamberlain et al. 2009 [\[14\]](#cit), Altenhoff et al. 1994 [\[15\]](#cit))

 



#### Bibliography

1. Baars,\ J.\ W.\ M.\ et\ al.\ 1977,\ A&A,\ 61,\ 99\ [ADS](http://adsabs.harvard.edu/abs/1977A%26A....61...99B)
2. Kellermann,\ K.\ I.\ 2009*,*\ A&A\ 500,\ 143\ [ADS](http://adsabs.harvard.edu/abs/2009A%26A...500..143K)
3. Perley,\ R.\ A.,\ &\ Butler,\ B.\ J.\ 2013,\ ApJS,\ 204,\ 19\ [ADS](http://adsabs.harvard.edu/abs/2013ApJS..204...19P)
4. Scaife,\ A.\ M.,\ &\ Heald,\ G.\ H.\ 2012,\ MNRAS,\ 423,\ 30\ [ADS](http://adsabs.harvard.edu/abs/2012MNRAS.423L..30S)
5. Partridge\ et\ al.\ 2016,\ ApJ\ 821,1\ [ADS](http://adsabs.harvard.edu/abs/2016ApJ...821...61P)
6. Clancy,\ R.T.\ et\ al.\ 2012,\ Icarus,\ 217,\ 779\ [ADS](http://adsabs.harvard.edu/abs/2012Icar..217..779C)
7. Perley,\ R.\ A.\ &\ Butler,\ B.\ J.\ 2017,\ ApJS,\ 230,7[ADS](http://adsabs.harvard.edu/abs/2017ApJS..230....7P)
7. Rudy,\ D.J.\ et\ al.\ 1987,\ Icarus,\ 71,\ 159\ [ADS](http://adsabs.harvard.edu/abs/1987Icar...71..159R)
8. Gurwell,\ M.A.\ &\ D.O.\ Muhleman\ 2000,\ Icarus,\ 145,\ 65w\ [ADS](http://adsabs.harvard.edu/abs/2000Icar..145..653G)
9. Gurwell,\ M.A.\ 2004,\ ApJ,\ 616,\ L7\ [ADS](http://adsabs.harvard.edu/abs/2004ApJ...616L...7G)
10. Moullet,\ A.\ et\ al.\ 2010,\ A&A,\ 516,\ L10\ [ADS](http://adsabs.harvard.edu/abs/2010A%26A...516L..10M)
11. Muller,\ T.G.\ &\ J.S.V.\ Lagerros\ 2002,\ A&A,\ 381,\ 324\ [ADS](http://adsabs.harvard.edu/abs/2002A%26A...381..324M) 
12. Redman,\ R.O.\ et\ al.\ 1998,\ AJ,\ 116,\ 1478\ [ADS](http://adsabs.harvard.edu/abs/1998AJ....116.1478R)
13. Altenhoff,\ W.J.\ et\ al.\ 1996,\ A&A,\ 309,\ 953\ [ADS](http://adsabs.harvard.edu/abs/1996A%26A...309..953A)
14. Chamberlain,\ M.A.\ et\ al.\ 2009,\ Icarus,\ 202,\ 487\ [ADS](http://adsabs.harvard.edu/abs/2009Icar..202..487C)
15. Altenhoff,\ W.J.\ et\ al.\ 1994,\ A&A,\ 287,\ 641\ [ADS](http://adsabs.harvard.edu/abs/1994A%26A...287..641A)
16. Leyrat,\ C.\ et\ al.\ 2012,\ A&A,\ 539,\ A154\ [ADS](http://adsabs.harvard.edu/abs/2012A%26A...539A.154L)



***





### Flux Calibrator Models - Data Formats 

Conventions and Data Formats

This section describes the conventions, the formats as well as the locations of the data of the flux density calibrator models used in setjy. The detailed descriptions of specific flux standards and list of the calibrators are found in [Flux Calibrator Models](memo-series.ipynb#flux-calibrator-models) in the Reference Material section.



#### Extragalactic flux calibrator source models

The spectral flux density models are expressed in a polynomial in the form$$\begin{equation} log S[Jy] = a + b*log\nu + c*log^2\nu + … \end{equation}$$where $\nu$ is a frequency either in MHz or GHz depending on the standard. In setjy, the point source model is constructed as a *componentlist* scaled by the spectral flux density model. For the standards, Baars, Perley 90, Perley-Taylor 95, Perley-Taylor 99, Perley-Butler 2010, and Stevens-Reynolds 2016,the polynomial coefficients are hard-coded in the code.

For Perley-Butler 2013 and Scaife-Heald 2012, the coefficients are stored in CASA tables called PerleyButler2013Coeffs and ScaifeHeald2012Coeffs, respectively located in \~/nrao/VLA/standards/ in the CASA data directory(from CASA prompt, you can find the data root path by typing casa\['dirs'\]\['data'\]). The separation of the data from the flux calibration code makes the maintenace easy and enable a user to acces the informaiton directly. Your can access these tables using the table tool (**tb**) and **browsetable** task. The list of the column header for PerleyButler2013Coeffs is shown below:

```python
CASA <8>: tb.colnames
--------> tb.colnames()
Out[8]:
['Epoch',
'3C48_coeffs',
'3C48_coefferrs',
'3C138_coeffs',
'3C138_coefferrs',
'3C147_coeffs',
'3C147_coefferrs',
'3C286_coeffs',
'3C286_coefferrs',
'3C123_coeffs',
'3C123_coefferrs',
'3C295_coeffs',
'3C295_coefferrs',
'3C196_coeffs',
'3C196_coefferrs']
```

The coefficients of each source are stored in a column as a vector and the corresponding errors are stored in a seperate column. The different row represents the corresponding coefficinets at that epoch for the time variable sources while for the steady sources each row contains identical information. The frequency is assumed in GHz.

The list of the column header for ScaifeHeald2012Coeffs is shown below:

```python
CASA <11>: tb.colnames
---------> tb.colnames()
Out[11]:
['Epoch',
'3C48_coeffs',
'3C48_coefferrs',
'3C147_coeffs',
'3C147_coefferrs',
'3C196_coeffs',
'3C196_coefferrs',
'3C286_coeffs',
'3C286_coefferrs',can b
'3C295_coeffs',
'3C295_coefferrs',
'3C380_coeffs',
'3C380_coefferrs']
```

 The reference frequnecy for Scaife-Heald 2012 is 150MHz. 

 



#### Solar System objects

For the solar system object used as a flux calibrator, **setjy** contstruct a model visibiity of the calibrator with the (averaged) brightness temperature model and ephemeris data of the sources as described in [ALMA Memo \#594](https://library.nrao.edu/public/memos/alma/memo594.pdf "ALMA Memo #594"). While the older Bulter-JPL-Horizons 2010 standard, hard-coded the brightness temperature models in the code, the models for Butler-JPL-Horizons 2012 are tabulated in ASCII files (SSobject_name_Tb.dat) located in the CASA data directory under \~/alma/SolarSystemModels. With an exception of Mars, the data for the brightness temparature models are stored in a simple format: 1st column - source frequency in GHz; 2nd column - the brightness temperature in Kelvin.  The follow example script shows how it can be plotted for Titan.

```
import numpy as np

rootdatapath=casa['dirs']['data']
source='Titan'
datapath=rootdatapath+'/alma/SolarSystemModels/'+source+'_Tb.dat'
data=np.genfromtxt(datapath)
data=data.transpose()

freq=data[0]
temp=data[1]
pl.plot(freq,temp)
pl.title(source+' Tb model')
pl.xlabel('Frequency (GHz)')
pl.ylabel('Tb (K)')

 
```

And the following is the output plot by executing the script above.

![be05be49165cd565e14b3473344dfb5d8c00c674](https://github.com/casangi/casadocs/blob/master/docs/notebooks/media/be05be49165cd565e14b3473344dfb5d8c00c674.png?raw=1){.image-inline}

 

The Tb model for Mars (Mars_Tb.dat) is calculated as a function of time and frequency, with tabulations every hour and at frequencies of: 30, 80, 115, 150, 200, 230, 260, 300, 330, 360, 425, 650, 800, 950, and 1000 GHz. The first line of the file contain frequencies in GHz. The data starts at the second line of the file with the format:    YYYY MM DD HH MJD Tb for at each frequency sepearated by a space.

 

**New Asteroid models**

Ceres_fd_time.dat, Luthetia_fd_time.dat, Pallas_fd_time.dat, and Vesta_fd_time.dat contain thermophysical models by Th. Mueller (private communication). These time variable models are already converted to flux densities and are tabulated for 30, 80, 115, 150, 200, 230, 260, 300, 330, 360, 425, 650, 800, 950, and 1000 GHz. Time intevals are 1 hr. for Ceres and 15min. for Luthetia, Pallas, and Vesta with the data available from 2015 01 01 0UT to 2021 01 01 0 UT.  In **setjy** task,these models are automatically selected for the data with the observation dates falls within this time range. 


***





### Spectral Frames 

Spectral Frames supported in CASA



#### Spectral Frames

CASA supported spectral frames:

  Frame            Description                                                                                                                                                                                                                                       Definition  
  ---------------- ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
  REST             rest frequency                                                                                                                                                                                                                                    Lab frame or source frame; cannot be converted to any other frame
  LSRK             LSR as a kinematic (radio) definition  (J2000) based on average velocity of stars in the Solar neighborhood                                                                                                                                       20km/s in direction of RA, Dec - \[270,+30\] deg (B1900.0) (Gordon 1975 [\[1\]](#Bibliography) )
  LSRD             Local Standard of Rest (J2000), dynamical, IAU definition. Solar peculiar velocity in the reference frame of a circular orbit about the Galactic Center, based on average velocity of stars in the Solar neighborhood and solar peculiar motion   U$\odot$=9kms/s, V$\odot$=12km/s,W$\odot$=7km/s. Or 16.552945km/s  towards l,b = 53.13, +25.02 deg (Delhaye 1965 [\[2\]](#Bibliography))
  BARY             Solar System Baryceneter (J2000)                                                                                                                                                                                                                   
  GEO              Geocentric, referenced to the Earth\'s center                                                                                                                                                                                                      
  TOPO             Topocentric                                                                                                                                                                                                                                       Local observatory frame, fixed in observing frequency, no doppler tracking
  GALACTO          Galactocentric (J2000), referenced to dynamical center of the Galaxy                                                                                                                                                                              220 km/s in the direction l,b = 270, +0 deg.  (Kerr and Lynden-Bell 1986  [\[3\]](#Bibliography))
  LGROUP           Mean motion of Local Group Galaxies with respect to its bary center                                                                                                                                                                               308km/s towards l,b = 105,-7
  CMB              Cosmic Microwave Background, COBE measurements of dipole anisotropy                                                                                                                                                                               369.5km/s towards l,b = 264.4,48.4. (Kogut et al. 1993 [\[4\]](#Bibliography))
  Undefined                                                                                                                                                                                                                                                           

 

 

 

 



####  Doppler Types 

CASA supported Doppler types (velocity conventions) where $f_v$ is the observed frequency and $f_0$ is the rest frame frequency of a given lineand positive velocity V is increasing away from the observer:

  Name           Description
  -------------- -----------------------------------------------------------------------------------------------------------------------------------
  RADIO          $$V = c \frac{(f_0 - f_v)}{f_0}$$
  Z              $$V=cz$$  $$z = \frac{(f_0 - f_v)}{f_v}$$
  RATIO          $$V=c(\frac{f_v}{f_o})$$
  BETA           $$V=c\frac{(1-(\frac{f_v}{f_0})^2)}{(1+(\frac{f_v}{f_0})^2)}$$
  GAMMA          $$ V=c\frac{(1 + (\frac{f_v}{f_0})^2)}{2\frac{f_v}{f_0}}$$
  OPTICAL        $$V= c\frac{(f_0 - f_v)}{f_v}$$
  TRUE           $$V=c\frac{(1-(\frac{f_v}{f_0})^2)}{(1+(\frac{f_v}{f_0})^2)}$$
  RELATIVISTIC   $$V=c\frac{(1-(\frac{f_v}{f_0})^2)}{(1+(\frac{f_v}{f_0})^2)}$$

 



#### Bibliography

1. Gordon\ 1975:\ \"**Methods\ of\ Experimental\ Physics:\ Volume\ 12:**\ Astrophysics,\ Part\ C:\ Radio\ Observations\",\ ed.\ M.L.Meeks,\ Academic\ Press\ 1976\ 
2. Delhaye\ 1965\ (
3. Kerr\ F.\ J.\ &\ Lynden-Bell\ D.\ 1986\ MNRAS,\ 221,\ 1023\ (
4. Kogut\ A.\ et\ al.\ 1993\ (
^



***





### Time Reference Frames 

CASA supported time reference frames: 
 

<table><colgroup><col style="width: 33%" /><col style="width: 33%" /><col style="width: 33%" /></colgroup><thead><tr class="header"><th>Acronym</th><th>Name</th><th>Description </th></tr></thead><tbody><tr class="odd"><td>ET</td><td>Ephemeris Time</td><td> The time scale used prior to 1984 as the independent variable in gravitational theories of the solar system. In 1984, ET was replaced by dynamical time (see TDB, TT).</td></tr><tr class="even"><td>GAST</td><td>Greenwich Apparent Sidereal Time</td><td> The Greenwich hour angle of the true equinox <sup><a href="#fn1">[1]</a></sup> of date.</td></tr><tr class="odd"><td>GMST</td><td>Greenwich Mean Sidereal Time</td><td><p>The Greenwich hour angle of the mean equinox <sup><a href="#fn1">[1]</a></sup> of date, defined as the angular distance on the celestial sphere measured westward along the celestial equator from the Greenwich meridian to the hour circle that passes through a celestial object or point.</p><p>GMST (in seconds at UT1=0) = 24110.54841 + 8640184.812866 * T<br />
+ 0.093104 * T<span class="MathJax_Preview" style="color: inherit;"></span><span class="math inline"><sup>2</sup></span> - 0.0000062 * T<span class="MathJax_Preview" style="color: inherit;"></span><span class="math inline"><sup>3</sup></span><br />
where T is in Julian centuries from 2000 Jan. 1 12h UT1:<br />
T = d / 36525<br />
d = JD - 2451545.0</p><p>(<a href="http://www.cv.nrao.edu/~rfisher/Ephemerides/times.html" class="external-link">http://www.cv.nrao.edu/~rfisher/Ephemerides/times.html</a>)</p></td></tr><tr class="even"><td>GMST1</td><td> </td><td> GMST calculated specifically with reference to UT1</td></tr><tr class="odd"><td>IAT</td><td>International Atomic Time (a.k.a. TAI en Francais):</td><td> The continuous time scale resulting from analysis by the Bureau International des Poids et Mesures of atomic time standards in many countries. The fundamental unit of TAI is the SI second <sup><a href="#fn2">[2]</a></sup> on the geoid <sup><a href="#fn3">[3]</a></sup> , and the epoch is 1958 January 1.</td></tr><tr class="even"><td>LAST</td><td>Local Apparent Sidereal Time</td><td><p>LAST is derived from LMST by applying the equation of equinoxes <sup><a href="#fn1">[1]</a></sup> or nutation of the mean pole of the Earth from mean to true position yields LAST.</p><p><a href="http://tycho.usno.navy.mil/sidereal.html" class="external-link">http://tycho.usno.navy.mil/sidereal.html</a></p></td></tr><tr class="odd"><td>LMST</td><td>Local Mean Sidereal Time</td><td><p>Sidereal time is the hour angle of the vernal equinox, the ascending node of the ecliptic on the celestial equator. The daily motion of this point provides a measure of the rotation of the Earth with respect to the stars, rather than the Sun. It corresponds to the coordinate right ascension of a celestial body that is presently on the local meridian.<br />
LMST is computed from the current GMST plus the local offset in longitude measured positive to the east of Greenwich, (converted to a sidereal offset by the ratio 1.00273790935 of the mean solar day to the mean sidereal day.)<br />
LMST = GMST + (observer's east longitude)</p><p><a href="http://www.cv.nrao.edu/~rfisher/Ephemerides/times.html" class="external-link">http://www.cv.nrao.edu/~rfisher/Ephemerides/times.html</a><br />
<a href="http://tycho.usno.navy.mil/sidereal.html" class="external-link">http://tycho.usno.navy.mil/sidereal.html</a></p></td></tr><tr class="even"><td>TAI</td><td>International Atomic Time (a.k.a. TAI en Francais)</td><td>see IAT</td></tr><tr class="odd"><td>TCB</td><td>Barycentric Coordinate Time</td><td> The coordinate time of the Barycentric Celestial Reference System (BCRS), which advances by SI seconds <sup><a href="#fn2">[2]</a></sup> within that system. TCB is related to TCG and TT by relativistic transformations that include a secular term.</td></tr><tr class="even"><td>TCG</td><td> </td><td> </td></tr><tr class="odd"><td>TDB</td><td>Barycentric Dynamical Time</td><td>A time scale defined by the IAU (originally in 1976; named in 1979; revised in 2006) used in barycentric ephemerides and equations of motion. TDB is a linear function of TCB that on average tracks TT over long periods of time; differences between TDB and TT evaluated at the Earth's surface remain under 2 ms for several thousand years around the current epoch. TDB is functionally equivalent to Teph, the independent argument of the JPL planetary and lunar ephemerides DE405/LE405.</td></tr><tr class="even"><td>TDT</td><td>Terrestrial Dynamical Time</td><td> The time scale for apparent geocentric ephemerides defined by a 1979 IAU resolution. In 1991 it was replaced by TT.</td></tr><tr class="odd"><td>TT</td><td>Terrestrial Time</td><td> An idealized form of International Atomic Time (TAI) with an epoch offset; in practice TT = TAI + 32s.184. TT thus advances by SI seconds on the geoid <sup><a href="#fn3">[3]</a></sup></td></tr><tr class="even"><td>UT</td><td>Universal Time</td><td> Loosely, mean solar time on the Greenwich meridian (previously referred to as Greenwich Mean Time). In current usage, UT refers either to UT1 or to UTC.</td></tr><tr class="odd"><td>UT1</td><td> </td><td> UT1 is formally defined by a mathematical expression that relates it to sidereal time. Thus, UT1 is observationally determined by the apparent diurnal motions of celestial bodies, and is affected by irregularities in the Earth's rate of rotation.</td></tr><tr class="even"><td>UT2</td><td> </td><td> <p>Before 1972 the time broadcast services kept their time signals within 0.1 seconds <sup><a href="#fn2">[2]</a></sup> of UT2, which is UT1 with annual and semiannual variations in the earth's rotation removed. The formal relation between UT1 and UT2 is</p><p>UT2 = UT1 + 0.022 * sin(2 * Pi * t) - 0.012 * cos(2 * Pi * t)</p><ul><li>0.006 * sin(4 * Pi * t) + 0.007 * cos(4 * Pi * t)<br />
where<br />
t = 2000.0 + (MJD - 51544.03) / 365.2422<br />
is the Besselian day fraction, and MJD is the Modified Julian Date (Julian Date - 2400000.5)<br />
<a href="http://www.cv.nrao.edu/~rfisher/Ephemerides/times.html" class="external-link">http://www.cv.nrao.edu/~rfisher/Ephemerides/times.html</a></li></ul></td></tr><tr class="odd"><td>UTC</td><td>Coordinated Universal Time</td><td>UTC is based on IAT but is maintained within 0s.9 of UT1 by the introduction of leap seconds when necessary.  </td></tr></tbody></table>

<table class="footnote-table" style="height: 217px;" width="828"><colgroup><col style="width: 50%" /><col style="width: 50%" /></colgroup><tbody><tr class="odd"><td style="text-align: left;">Footnote Number</td><td>1</td></tr><tr class="even"><td style="text-align: left;">Footnote Text</td><td><p>mean equator and equinox v. true equator and equinox: The mean equator and equinox are used for the celestial coordinate system defined by the orientation of the Earth's equatorial plane on some specified date together with the direction of the dynamical equinox on that date, neglecting nutation. Thus, the mean equator and equinox moves in response only to precession. Positions in a star catalog have traditionally been referred to a catalog equator and equinox that approximate the mean equator and equinox of a standard epoch.<br />
The true equator and equinox are affected by both precession and nutation. The Equation of the Equinoxes is the difference (apparent sidereal time minus mean sidereal time). Equivalently, the difference between the right ascensions of the true and mean equinoxes, expressed in time units.</p><p> </p><p> </p><p> </p></td></tr></tbody></table>

<table class="footnote-table"><colgroup><col style="width: 50%" /><col style="width: 50%" /></colgroup><tbody><tr class="odd"><td style="text-align: left;"><p>FootnoteNumber</p></td><td>2</td></tr><tr class="even"><td style="text-align: left;">Footnote Text</td><td><p> The Systeme International (SI) second is defined as the duration of 9,192,631,770 cycles of radiation corresponding to the transition between two hyperfine levels of the ground state of caesium 133.</p><p> </p></td></tr></tbody></table>

<table class="footnote-table"><colgroup><col style="width: 50%" /><col style="width: 50%" /></colgroup><tbody><tr class="odd"><td style="text-align: left;">Footnote Number</td><td>3</td></tr><tr class="even"><td style="text-align: left;">Footnote Text</td><td><p>The geoid is an equipotential surface that coincides with mean sea level in the open ocean. On land it is the level surface that would be assumed by water in an imaginary network of frictionless channels connected to the ocean.</p><p> </p><p> </p></td></tr></tbody></table>

Footnote(s)

<div>

^1.\ mean\ equator\ and\ equinox\ v.\ true\ equator\ and\ equinox:\ The\ mean\ equator\ and\ equinox\ are\ used\ for\ the\ celestial\ coordinate\ system\ defined\ by\ the\ orientation\ of\ the\ Earth\'s\ equatorial\ plane\ on\ some\ specified\ date\ together\ with\ the\ direction\ of\ the\ dynamical\ equinox\ on\ that\ date,\ neglecting\ nutation.\ Thus,\ the\ mean\ equator\ and\ equinox\ moves\ in\ response\ only\ to\ precession.\ Positions\ in\ a\ star\ catalog\ have\ traditionally\ been\ referred\ to\ a\ catalog\ equator\ and\ equinox\ that\ approximate\ the\ mean\ equator\ and\ equinox\ of\ a\ standard\ epoch.\ The\ true\ equator\ and\ equinox\ are\ affected\ by\ both\ precession\ and\ nutation.\ The\ Equation\ of\ the\ Equinoxes\ is\ the\ difference\ (apparent\ sidereal\ time\ minus\ mean\ sidereal\ time).\ Equivalently,\ the\ difference\ between\ the\ right\ ascensions\ of\ the\ true\ and\ mean\ equinoxes,\ expressed\ in\ time\ units.\  \  \  \ [↩](#ref1 "Jump back to footnote 1 in the text.")^

</div>

<div>

^2.\  The\ Systeme\ International\ (SI)\ second\ is\ defined\ as\ the\ duration\ of\ 9,192,631,770\ cycles\ of\ radiation\ corresponding\ to\ the\ transition\ between\ two\ hyperfine\ levels\ of\ the\ ground\ state\ of\ caesium\ 133.\  \ [↩](#ref2 "Jump back to footnote 2 in the text.")^

</div>

<div>

^3.\ The\ geoid\ is\ an\ equipotential\ surface\ that\ coincides\ with\ mean\ sea\ level\ in\ the\ open\ ocean.\ On\ land\ it\ is\ the\ level\ surface\ that\ would\ be\ assumed\ by\ water\ in\ an\ imaginary\ network\ of\ frictionless\ channels\ connected\ to\ the\ ocean.\  \  \ [↩](#ref3 "Jump back to footnote 3 in the text.")^

</div>



***





### Coordinate Frames 


CASA supported spatial coordinate frames:

  

  Name        Description
  ----------- -------------------------------------------------
  J2000       mean equator and equinox at J2000.0 (FK5)
  JNAT        geocentric natural frame
  JMEAN       mean equator and equinox at frame epoch
  JTRUE       true equator and equinox at frame epoch
  APP         apparent geocentric position
  B1950       mean epoch and ecliptic at B1950.0.
  B1950_VLA   mean epoch (1979.9) and ecliptic at B1950.0
  BMEAN       mean equator and equinox at frame epoch
  BTRUE       true equator and equinox at frame epoch
  GALACTIC    Galactic coordinates
  HADEC       topocentric HA and declination
  AZEL        topocentric Azimuth and Elevation (N through E)
  AZELSW      topocentric Azimuth and Elevation (S through W)
  AZELNE      topocentric Azimuth and Elevation (N through E)
  AZELGEO     geodetic Azimuth and Elevation (N through E)
  AZELSWGEO   geodetic Azimuth and Elevation (S through W)
  AZELNEGEO   geodetic Azimuth and Elevation (N through E)
  ECLIPTC     ecliptic for J2000 equator and equinox
  MECLIPTIC   ecliptic for mean equator of date
  TECLIPTIC   ecliptic for true equator of date
  SUPERGAL    supergalactic coordinates
  ITRF        coordinates wrt ITRF Earth frame
  TOPO        apparent topocentric position
  ICRS        International Celestial reference system



***





### Physical Units 

CASA regonizes physical units as listed in the following Tables.

  Prefix   Name      Value
  -------->Physical units: Prefixes
  

  

  Unit     Name                  Value
  -------- --------------------- --------------------------
  \$       (currency)            1 \_
  \%       (percent)             0.01
  %%       (permille)            0.001
  A        (ampere)              1 A
  AE       (astronomical unit)   149597870659 m
  AU       (astronomical unit)   149597870659 m
  Bq       (becquerel)           1 s^−1^
  C        (coulomb)             1 s A
  F        (farad)               1 m^−2^ kg^−1^ s^4^ A^2^
  Gy       (gray)                1 m^2^ s^−2^
  H        (henry)               1 m^2^ kg s^−2^ A^−2^
  Hz       (hertz)               1 s^−1^
  J        (joule)               1 m^2^ kg s^−2^
  Jy       (jansky)              10^−26^ kg s^−2^
  K        (kelvin)              1 K
  L        (litre)               0.001 m^3^
  M0       (solar mass)          1.98891944407× 10^30^ kg
  N        (newton)              1 m kg s^−2^
  Ohm      (ohm)                 1 m^2^ kg s^−3^ A^−2^
  Pa       (pascal)              1 m^−1^ kg s^−2^
  S        (siemens)             1 m^−2^ kg^−1^ s^3^ A^2^
  S0       (solar mass)          1.98891944407× 10^30^ kg
  Sv       (sievert)             1 m^2^ s^−2^
  T        (tesla)               1 kg s^−2^ A^−1^
  UA       (astronomical unit)   149597870659 m
  V        (volt)                1 m^2^ kg s^−3^ A^−1^
  W        (watt)                1 m^2^ kg s^−3^
  Wb       (weber)               1 m^2^ kg s^−2^ A^−1^
  \_       (undimensioned)       1 \_
  a        (year)                31557600 s
  arcmin   (arcmin)              0.000290888208666 rad
  arcsec   (arcsec)              4.8481368111×10^−6^ rad
  as       (arcsec)              4.8481368111e×10^−6^ rad
  cd       (candela)             1 cd
  cy       (century)             3155760000 s
  d        (day)                 86400 s
  deg      (degree)              0.0174532925199 rad
  g        (gram)                0.001 kg
  h        (hour)                3600 s
  l        (litre)               0.001 m^3^
  lm       (lumen)               1 cd sr
  lx       (lux)                 1 m^−2^ cd sr
  m        (metre)               1 m
  min      (minute)              60 s
  mol      (mole)                1 mol
  pc       (parsec)              3.08567758065×10^16^ m
  rad      (radian)              1 rad
  s        (second)              1 s
  sr       (steradian)           1 sr
  t        (tonne)               1000 kg

>Physical SI Units
  

 

  Unit        Name                           Value
  ----------- ------------------------------ ----------------------------------------------
  \"          (arcsec)                       4.8481368111×10^−6^ rad
  \"\_2       (square arcsec)                2.35044305391× 10^−11^ sr
  '           (arcmin)                       0.000290888208666 rad
  "           (arcsec)                       4.8481368111×10^−6^ rad
  "\_2        (square arcsec)                2.35044305391×10^−11^ sr
  '\_2        (square arcmin)                8.46159499408×10^−8^ sr
  :           (hour)                         3600 s
  ::          (minute)                       60 s
  
  Ah          (ampere hour)                  3600 s A
  Angstrom    (angstrom)                     1e-10 m
  Btu         (British thermal unit (Int))   1055.056 m^2^ kg s^−2^
  CM          (metric carat)                 0.0002 kg
  Cal         (large calorie (Int))          4186.8 m^2^ kg s^−2^
  FU          (flux unit)                    10^−26^ kg s^−2^
  G           (gauss)                        0.0001 kg s^−2^ A^−1^
  Gal         (gal)                          0.01 m s^−2^
  Gb          (gilbert)                      0.795774715459 A
  Mx          (maxwell)                      10^−8^ m^2^ kg s^−2^ A^−1^
  Oe          (oersted)                      79.5774715459 m^−1^ A
  R           (mile)                         0.000258 kg^−1^ s A
  St          (stokes)                       0.0001 m^2^ s−1
  Torr        (torr)                         133.322368421 m^−1^ kg s^−2^
  USfl_oz     (fluid ounce (US))             2.95735295625×10^−5^ m^3^
  USgal       (gallon (US))                  0.003785411784 m^3^
  WU          (WSRT flux unit)               5× 10^−29^ kg s^−2^
  abA         (abampere)                     10 A
  abC         (abcoulomb)                    10 s A
  abF         (abfarad)                      10^9^ m^−2^ kg^−1^ s^4^ A^2^
  abH         (abhenry)                      10^−9^ m^2^ kg s^−2^ A^−2^
  abOhm       (abohm)                        10^−9^ m^2^ kg s^−3^ A^−2^
  abV         (abvolt)                       10^−8^ m^2^ kg s^−3^ A^−1^
  ac          (acre)                         4046.8564224 m^2^
  arcmin_2    (square arcmin)                8.46-2159499408×10^−8^ sr
  arcsec_2    (square arcsec)                2.35044305391×10^−11^ sr
  ata         (technical atmosphere)         98066.5 m^−1^.kg.s^−2^
  atm         (standard atmosphere)          101325 m^−1^.kg.s^−2^
  bar         (bar)                          100000 m^−1^.kg.s^−2^
  beam        (undefined beam area)          1 \_
  cal         (calorie (Int))                4.1868 m^2^ kg s^−2^
  count       (count)                        1 \_
  cwt         (hundredweight)                50.80234544 kg
  deg_2       (square degree)                0.000304617419787 sr
  dyn         (dyne)                         10^−5^ m kg s^−2^
  eV          (electron volt)                1.60217733×10^−19^ m^2^ kg s^−2^
  erg         (erg)                          10^−7^ m^2^ kg s^−2^
  fl_oz       (fluid ounce (Imp))            2.84130488996×10^−5^ m^3^
  ft          (foot)                         0.3048 m
  fu          (flux unit)                    10^−26^ kg s^−2^
  fur         (furlong)                      201.168 m
  gal         (gallon (Imp))                 0.00454608782394 m^3^
  ha          (hectare)                      10000 m^2^
  hp          (horsepower)                   745.7 m^2^ kg s^−3^
  in          (inch)                         0.0254 m
  kn          (knot (Imp))                   0.514773333333 m s^−1^
  lambda      (lambda)                       1 \_
  lb          (pound (avoirdupois))          0.45359237 kg
  ly          (light year)                   9.46073047×10^15^ m
  mHg         (metre of mercury)             133322.387415 m^−1^ kg s^−2^
  mile        (mile)                         1609.344 m
  n_mile      (nautical mile (Imp))          1853.184 m
  oz          (ounce (avoirdupois))          0.028349523125 kg
  pixel       (pixel)                        1 \_
  sb          (stilb)                        10000 m^−2^ cd
  sq_arcmin   (square arcmin)                8.46159499408×10^−8^ sr
  sq_arcsec   (square arcsec)                2.35044305391×10^−11^ sr
  sq_deg      (square degree)                0.000304617419787 sr
  statA       (statampere)                   3.33564095198×10^−10^ A
  statC       (statcoulomb)                  3.33564095198×10^−10^ s A
  statF       (statfarad)                    1.11188031733×10^−12^ m^−2^ kg^−1^ s^4^ A^2^
  statH       (stathenry)                    899377374000 m^2^ kg s^−2^ A^−2^
  statOhm     (statohm)                      899377374000 m^2^ kg s^−3^ A^−2^
  statV       (statvolt)                     299.792458 m^2^ kg s^−3^ A^−1^
  u           (atomic mass unit)             1.661×10^−27^  kg
  yd          (yard)                         0.9144 m
  yr          (year)                         31557600 s

>Custom units available in CASA
  



***





### Physical Constants 

Physical constants recognized by CASA

The following physical constants are defined in CASA:

 

  Constant   Name                 Value
  ---------- -------------------- --------------------------------
  pi         3.14..               3.14159
  ee         2.71..               2.71828
  c          speed of light       2.99792×10^8^ m s^−1^
  G          grav. constant       6.67259×10^11^ N m^2^ kg^−2^
  h          Planck constant      6.62608×10^−34^ J s
  HI         HI line frequency    1420.41 MHz
  R          gas constant         8.31451 J K^−1^ mol^−1^
  NA         Avogadro \#          6.02214×10^23^ mol^−1^
  e          electron charge      1.60218×10^−19^ C
  mp         proton mass          1.67262×10^−27^  kg
  mp_me      mp/me                1836.15
  mu0        permeability vac.    1.25664×10^−6^ H m^−1^
  eps0       permittivity vac.    8.85418782×10^−12^ F m^-1^
  k          Boltzmann constant   1.38066×10^−23^ J K^−1^
  F          Faraday constant     96485.3 C mol^−1^
  me         electron mass        9.10939×10^−31^  kg
  re         electron radius      2.8179e×10^−15^  m
  a0         Bohrs radius         5.2918×10^−11^  m
  R0         solar radius         6.9599×10^8^  m
  k2         IAU grav. const^2^   0.000295912 AU^3^ d^−2^ S0^−1^

 



***





## Resources and Links 

Useful supplementary information relevant to CASA

 



### Tutorials

-   [CASAguides](https://casaguides.nrao.edu/) contains the most comprehensive data reduction tutorials for ALMA (including simulations) and the VLA. Tutorials also exist for other telescopes, such as ATCA, SMA, and CARMA. The [CASAguides](https://casaguides.nrao.edu/) wiki is also a great resource for tips and tricks in using CASA and contains user contributed scripts.



### Workshops

-   [ALMA community days](https://science.nrao.edu/facilities/alma/community1) webpages provide material that was used during workshops
-   The [2018 VLA data reduction workshop](http://www.cvent.com/events/16th-synthesis-imaging-workshop/event-summary-b36e4bc16b574d5d94229f9d885d0eff.aspx) contains course material
-   Synthesis Imaging in Radio Astronomy II ([pdf book](https://drive.google.com/file/d/1qOcQ1xR96LVPCssTu2IxYA3ItkGdVMKj/view?usp=sharing) \([chapters](http://www.aspbooks.org/a/volumes/table_of_contents/?book_id=292)\)



### Python

-   [Python documentation](https://casa.nrao.edu/python.shtml)



### Science Data Model

-   The SDM description and SDM definition document can be found on [this page](casa-fundamentals.ipynb#science-data-model)



### Dependencies/Libraries

-   [CASA package code libraries and dependencies](https://safe.nrao.edu/wiki/bin/view/Software/CASA/CASADevelopmentToolsAndLibraries)



### Memos

-   [ALMA](http://library.nrao.edu/alma.shtml)
-   [VLA](http://www.aoc.nrao.edu/evla/memolist.shtml)
-   [ngVLA](http://library.nrao.edu/ngvla.shtml)



### Related Telescopes

-   [ALMA](http://www.almascience.org/)
-   [VLA](https://public.nrao.edu/telescopes/vla/)
-   [Greenbank](https://greenbankobservatory.org/science/)
-   [VLBA](https://www.lbo.us/)
-   [ATCA](https://www.narrabri.atnf.csiro.au/)
-   [GMRT](http://www.ncra.tifr.res.in/ncra/gmrt)
-   [Nobeyama](http://www.nro.nao.ac.jp/~nro45mrt/html/index-e.html)
-   [SMA](https://www.cfa.harvard.edu/sma/)
-   [NOEMA](http://iram-institute.org/EN/noema-project.php)
-   [MeerKAT](https://www.ska.ac.za/gallery/meerkat/)
-   [Next Generation VLA](http://ngvla.nrao.edu/)
-   [Square Kilometre Array](https://www.skatelescope.org/)